In [227]:
import numpy as np
import pandas as pd
pd.set_option('display.max.columns', 100)
# to draw pictures in jupyter notebook
%matplotlib inline 
import matplotlib.pyplot as plt
import seaborn as sns
# we don't like warnings
# you can comment the following 2 lines if you'd like to
import warnings
warnings.filterwarnings('ignore')

In [228]:
data = pd.read_csv('data/abult.data.csv')
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,=50K


1) Сколько мужчин и женщин в наборе данных?

In [229]:
data['sex'].value_counts()

 Male      21790
 Female    10771
Name: sex, dtype: int64

2) Какой средний возраст женщин?

In [230]:
data.loc[data['sex'] == ' Female', 'age'].mean()

36.85823043357163

3) Доля граждан из Германии?

In [231]:
 (float((data['native-country'] == ' Germany').sum())/data.shape[0] * 100)

0.42074874850281013

4-5) Каково среднее значение и стандартное отклонение возраста тех, кто получает более 50 тыс. В год и тех, кто получает менее 50 тыс. В год?

In [232]:
people_1 = data.loc[data['salary'] == ' 50K', 'age']
people_2 = data.loc[data['salary'] == ' =50K', 'age']
print('Средний возраст первой группы: {0} +- {1} years, \
      \nвторой группы: {2} +- {3} years'.format(round(people_1.mean()), round(people_1.std(), 2), round(people_2.mean()), round(people_2.std(), 2) ))

Средний возраст первой группы: 44 +- 10.52 years,       
второй группы: 37 +- 14.02 years


6) Правда ли, что люди, которые более 50 тысяч, имеют хотя бы среднее образование?

In [233]:
high_schools= [' Bachelors', ' Prof-school', ' Assoc-acdm', ' Assoc-voc', ' Masters or Doctorate feature']
flag = 'Нет'
for hl in high_schools:
    if hl in data.loc[data['salary'] == ' 50K', 'education'].unique():
        flag='Да' 
flag

'Да'

7) Отобразите статистику возраста для каждой расы и каждого пола. Нати максимальный возраст мужчин американо-индийско-эскимосской расы.

In [234]:
data[(data['race']==' Amer-Indian-Eskimo') & (data['sex']==' Male')].max()['age']

82

In [235]:
for (race, sex), df in data.groupby(['race', 'sex']):
    print('Race: {} ,Sex: {}'.format(race, sex))
    print(df['age'].describe())

Race:  Amer-Indian-Eskimo ,Sex:  Female
count    119.000000
mean      37.117647
std       13.114991
min       17.000000
25%       27.000000
50%       36.000000
75%       46.000000
max       80.000000
Name: age, dtype: float64
Race:  Amer-Indian-Eskimo ,Sex:  Male
count    192.000000
mean      37.208333
std       12.049563
min       17.000000
25%       28.000000
50%       35.000000
75%       45.000000
max       82.000000
Name: age, dtype: float64
Race:  Asian-Pac-Islander ,Sex:  Female
count    346.000000
mean      35.089595
std       12.300845
min       17.000000
25%       25.000000
50%       33.000000
75%       43.750000
max       75.000000
Name: age, dtype: float64
Race:  Asian-Pac-Islander ,Sex:  Male
count    693.000000
mean      39.073593
std       12.883944
min       18.000000
25%       29.000000
50%       37.000000
75%       46.000000
max       90.000000
Name: age, dtype: float64
Race:  Black ,Sex:  Female
count    1555.000000
mean       37.854019
std        12.637197
min       

**8) Среди кого больше доля тех,кто много зарабатывает (> 50 тыс.) замужние или одинокие мужчины? Считается, что в браке находятся (Married-civ-spouse, Married-spouse-absent, Married-AF-spouse).**

In [236]:
x_mar, y_mar = data.loc[((data['sex'] == ' Male') &
     (data['marital-status'].str.startswith(' Married'))), 'salary'].value_counts()
#data['marital-status'].isin([])
x_no_mar, y_no_mar = data.loc[((data['sex'] == ' Male') & (~data['marital-status'].str.startswith(' Married'))), 'salary'].value_counts()
mar, no_mar=y_mar/(x_mar+y_mar)*100, y_no_mar/(x_no_mar+y_no_mar)*100
if mar > no_mar:
    print('Больше среди замужних')
else:
    print('Больше среди одиноких') 
print('Процент тех кто много зарабатывает:\nЗамужние: {}%\nОдинокие: {}%'.format(mar, no_mar))

Больше среди замужних
Процент тех кто много зарабатывает:
Замужние: 44.05139945351156%
Одинокие: 8.449509031397746%


**9) Какое максимальное количество часов в неделю работает человек? Сколько человек работает столько часов, и какой процент тех, кто зарабатывает много (>50) среди них?**

In [237]:
max_load = data['hours-per-week'].max()
num_max_all = (data['hours-per-week'] == 99).sum()
rich = int(data[(data['hours-per-week'] == 99) & (data['salary']==' 50K')].shape[0]/num_max_all*100)
print("Максимальная занятость: {} час/неделя\nКоличество человек на такой занятости: {}\nИз них процент богатых: {}%".format(max_load, num_max_all, rich))

Максимальная занятость: 99 час/неделя
Количество человек на такой занятости: 85
Из них процент богатых: 29%


**10) Среднее время работы тех, кто зарабатывает мало и много в каждой из стран.**


In [238]:
pd.crosstab(data['native-country'], data['salary'], 
           values=data['hours-per-week'], aggfunc=np.mean).T

native-country,,Cambodia,Canada,China,Columbia,Cuba,Dominican-Republic,Ecuador,El-Salvador,England,France,Germany,Greece,Guatemala,Haiti,Holand-Netherlands,Honduras,Hong,Hungary,India,Iran,Ireland,Italy,Jamaica,Japan,Laos,Mexico,Nicaragua,Outlying-US(Guam-USVI-etc),Peru,Philippines,Poland,Portugal,Puerto-Rico,Scotland,South,Taiwan,Thailand,Trinadad&Tobago,United-States,Vietnam,Yugoslavia
salary,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
50K,45.547945,40.000000,45.641026,38.900000,50.000000,42.440000,47.000000,48.750000,45.000000,44.533333,50.750000,44.977273,50.625000,36.666667,42.750,NaN,60.000000,45.000000,50.0,46.475000,47.50,48.000000,45.400,41.100000,47.958333,40.000,46.575758,37.50000,NaN,40.000000,43.032787,39.000000,41.500000,39.416667,46.666667,51.43750,46.800000,58.333333,40.000000,45.505369,39.200000,49.5
=50K,40.164760,41.416667,37.914634,37.381818,38.684211,37.985714,42.338235,38.041667,36.030928,40.483333,41.058824,39.139785,41.809524,39.360656,36.325,40.0,34.333333,39.142857,31.3,38.233333,41.44,40.947368,39.625,38.239437,41.000000,40.375,40.003279,36.09375,41.857143,35.068966,38.065693,38.166667,41.939394,38.470588,39.444444,40.15625,33.774194,42.866667,37.058824,38.799127,37.193548,41.6


In [239]:
import pandasql as ps
import time

In [240]:
def example1_pandasql(data):
    simple_query = '''
        SELECT 
            avg(age)
        FROM data
        GROUP BY sex
        '''
    return ps.sqldf(simple_query, locals())

In [241]:
pandasql_time_start = time.time()
a = example1_pandasql(data)
print (a)
pandasql_time = - pandasql_time_start + time.time()

    avg(age)
0  36.858230
1  39.433547


In [242]:
pandas_time_start = time.time()
data.groupby('sex')['age'].mean()
pandas_time = - pandas_time_start + time.time()

In [243]:
print('PandaSQL: {}\nPandas: {}'.format(pandasql_time, pandas_time))

PandaSQL: 0.5626976490020752
Pandas: 0.003999233245849609


In [251]:
d1 = data[['age','workclass']].head(10)
d2 = data[['age','fnlwgt']].head(10)
d1

,age,workclass
0,39,State-gov
1,50,Self-emp-not-inc
2,38,Private
3,53,Private
4,28,Private
5,37,Private
6,49,Private
7,52,Self-emp-not-inc
8,31,Private
9,42,Private


In [252]:
def example2_pandasql(data):
    simple_query = '''
        SELECT 
            d.age, 
            workclass,
            education
        FROM (SELECT age,workclass FROM data) d, (SELECT age,education FROM data) d2
        '''
    return ps.sqldf(simple_query, locals())

In [ ]:
pandasql_time_start = time.time()
a = example2_pandasql(data)
print (a)
pandasql_time = - pandasql_time_start + time.time()